# DICE — Notebook 2 (Exercises): Scenarios & Forcing


**Learning objectives (1h30min)**
0. Load a minimal DICE code and identify its **blocks**: parameters, initial states, economic equations, climate equations.  
1. SSP Scenarios: generate meaningful SSP scenarios by adjusting the calibration of exogenous variables.
2. Introduce some forcing objectives, by adjusting

> This notebook uses the provided `DICE.py` module (teaching version).


# 0) Load DICE and run baseline

In [9]:

# 1) Load DICE and baseline run
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import DICE 

p = Params()
path = init_states(p)
path[:, p.i_mu] = 0.03          # baseline abatement path
path = update_path(path, range(1, p.nT), p)
df_base = mat_to_df(path, p)
df_base.head()


,time,A,s,Y,Q,C,K,L,sigma,theta1,...,E,F,T_AT,T_LO,E_land,F_EX,M_AT,M_UP,M_LO,Tax
0,2015.0,5.115000,NaN,NaN,NaN,NaN,220.000000,7403.000000,0.350300,0.074102,...,NaN,NaN,0.850000,0.007000,NaN,NaN,851.000000,460.000000,1740.000000,NaN
1,2020.0,5.524495,0.25,116.907383,583.537630,437.653222,275.792207,7757.941083,0.324664,0.066962,...,195.589590,2.322138,0.989258,0.028075,2.301000,0.529412,892.382616,471.289302,1740.670698,0.535168
2,2025.0,5.955003,0.25,138.742156,692.107702,519.080777,335.879466,8079.041057,0.300904,0.060510,...,212.659791,2.581809,1.139229,0.052105,2.036385,0.558824,935.667530,485.253773,1741.419438,0.521788
3,2030.0,6.406732,0.25,162.300802,809.021177,606.765883,400.588760,8367.832264,0.278882,0.054679,...,228.535707,2.840911,1.298675,0.079283,1.802201,0.588235,980.825086,501.578742,1742.264832,0.508744
4,2035.0,6.879842,0.25,187.699735,934.772932,701.079699,470.236890,8626.253064,0.258473,0.049411,...,243.273701,3.098891,1.466438,0.109767,1.594948,0.617647,1027.782882,520.009889,1743.223263,0.496025


## 1) SSP Scenarios

In what follows, we want to simulate two extreme narratives, the **SSP1** and **SSP5** storylines, as well as the **baseline (SSP2)**.  

- **SSP1** – *Sustainability: Taking the Green Road (Low challenges to mitigation and adaptation)*  
  *The world shifts gradually toward a more sustainable path, emphasizing inclusive development and respect for environmental boundaries. Educational and health investments accelerate the demographic transition, inequality is reduced, and consumption patterns shift toward low material growth and lower resource and energy intensity.*

- **SSP5** – *Fossil-fueled Development: Taking the Highway (High challenges to mitigation, low challenges to adaptation)*  
  *This world emphasizes competitive markets, innovation, and human capital development. Rapid economic growth is fueled by fossil resources and energy-intensive lifestyles. Local environmental problems are managed, but global commons are stressed, with high emissions and reliance on future technological fixes or geo-engineering.*

We thus consider three scenarios: the **optimistic** (SSP1), the **baseline** (SSP2, current calibration), and the **pessimistic** (SSP5).  
The table below summarizes the proposed core calibration changes:

| Parameter (description)                | Symbol   | SSP1 (Optimistic)                         | SSP2 (Baseline — DICE) | SSP5 (Pessimistic)                        |
|----------------------------------------|----------|--------------------------------------------|-------------------------|-------------------------------------------|
| Emissions intensity (initial)          | $\sigma_0$ | 25% lower than baseline                     | calibrated value        | 15% higher than baseline                   |
| Decline rate of emissions intensity    | $g_{\sigma}$ | 25% faster decline                         | calibrated value        | 50% slower decline                        |
| Long-run population (asymptote)        | $L_{\infty}$ | 9 billion                                  | calibrated value        | 16 billion                                |
| Population growth parameter            | $l_g$      | 20% lower                           | calibrated value        | 30% higher                        |
| Total factor productivity (TFP) level  | $A_0$      | 5% higher than baseline                    | calibrated value        | 3% lower than baseline                    |
| TFP growth rate                        | $g_A$      | 15% faster than baseline                   | calibrated value        | 20% slower than baseline                  |

> ✍️ **Instructions**: For each scenario, initialize parameters accordingly, run the model, and compare trajectories for output, emissions, forcing, and temperatures.


#### 1-A) Build the calibration and check it works in new p vector.
Use the table to fill each scenarios and have the right narrative.

In [20]:
# Your code for question 1-A should be here

def make_params_variant(**kwargs):
    q = Params()
    for k,v in kwargs.items():
        setattr(q, k, v)
    return q

p_base = Params()
p_opt  = make_params_variant(A0=p_base.A0*1, gA=p_base.gA*1.15)
p_pes  = make_params_variant(A0=p_base.A0*1, gA=p_base.gA*0.80)


#### 1-B) Provide some graph comparing Emissions, GDP, damages (define them manually) and temperatures.

In [13]:
# Your code for question 1-B should be here

### 1-B) Report the graph and compare them
sim_bas = init_states(p_base)
#sim_opt = init_states(p_opt)
#sim_pes = init_states(p_pes)


sim_bas[1:, p_base.i_s]  = 0.20  # constant saving rate at 20%
sim_bas[1:, p_base.i_mu] = 0.03
timevec = range(1, p.nT)
sim_bas = update_path(sim_bas, timevec, p_base)


#### 1-C) According to all the figures, do you think that SSPs narrative are the main drivers of climate change? What is the temperatures in best versus worst case of climate change?

> ✍️ You written answer here.

## 2) Mitigation schedules: ramp abatement to reach (near) net-zero by 2100 / 2080 / 2060

Up to now, our scenarios were purely **exogenous**: we varied productivity, population, and emissions intensity to mimic different SSP narratives.  
We now enrich the analysis by introducing a **second decision variable**: the **abatement rate** $\mu_t$, which measures the fraction of industrial emissions avoided through mitigation policies and technologies.  

- In the **baseline** exercises, $\mu_t$ was fixed at a constant, low level (e.g. 3%).  
- In this exercise, we explicitly feed an **abatement trajectory** $\{\mu_t\}$ into the model.  
- Conceptually, pushing $\mu_t \to 1$ means that almost all industrial CO₂ emissions are eliminated. Note, however, that **land-use emissions** $E_{\text{land},t}$ remain exogenous and may stay positive for some time.

This mirrors the way scenarios are constructed in **CMIP6 (Coupled Model Intercomparison Project Phase 6)**, where each SSP storyline can be combined with different **climate policy assumptions**:  
- A **high-ambition mitigation pathway** ramps abatement quickly, reaching **net-zero emissions** by mid-century (e.g. 2060).  
- A **delayed mitigation pathway** reaches net-zero much later (e.g. 2100).  
- The resulting **forcing levels by 2100** (e.g. 2.6, 4.5, 8.5 W/m²) define the well-known **SSP1-2.6**, **SSP2-4.5**, and **SSP5-8.5** scenarios.  


3. Interpret the results in light of the IPCC’s SSP scenario framework: *early and ambitious decarbonization (SSP1-2.6) vs. delayed action with high warming (SSP5-8.5).*  

#### 2-a) Construct linear ramps for $\mu_t$ that reach **net zero** by 2100, 2080, and 2060. 
Recall that in this model, we only have two decision variables in here. Saving remains unchanged $\{s_t\}_{t=t_{0}}^{T}=0.2$ while mitigation path must be adjusted: $\{\mu_t\}_{t=t_{0}}^{T}$. It should gradually increase to 1 to reach net zero at some specific date.
Recall that here, our simulations, including decision variables, are all stacked into matrix $w = [y,x,z]$. In what follows, we modify directly $w$.



In [ ]:
# Your answer here
# sim = init_states(p_mit)
#sim[:, p.i_mu] = ...
# sim = update_path(sim,...


#### 2-b) Simulate the model under each ramp and compare the paths for **forcing $F_t$**, **temperatures $T_{AT,t}$**, and **emissions $E_t$**.
Recall that once the matrix of decision $z$ of size $I\times 2$ has been modified (as part of matri $w$), we can once again solve the new system with alternative climate policy:
  $$
  y_t = f_p (y_{t-1}, x_{t-1}, z_t), \quad f: \mathbb{R}^{N_y}\times \mathbb{R}^{N_x}\times \mathbb{R}^2 \to \mathbb{R}^{N_y}.
  $$

  How effective is climate policy in curbing emissions?

In [ ]:
# Your code here.

> ✍️ You written answer here.

### 2-c) What is the economic cost of climate mitigation? 
Provide a 2 plots, one showing the consumption gap defined as $100\times (c_t^{alt}/c_t^{base}-1)$ and in a second figure the temperature gap  $(T_t^{alt}-T_t^{base})$. $T_t^{alt}$ denotes the temperature path in alternative regimes (e.g. optimistic or pessimistic) while *base* is baseline.  
Comment the result your find about the sacrifice that must be done by society in terms of consumption loss during the transition. What drives this sacrifice?

In [ ]:
# Your code here.

> ✍️ You written answer here.

## 3) Providing core scenarios mixing SSPs $\times$ Forcing as in IPCC


The table below shows how **SSP storylines** combine with **radiative forcing targets** (W/m² by 2100).  
For example, *SSP1-2.6* corresponds to the **SSP1 narrative** combined with a **2.6 W/m² forcing pathway**.

|      | **1.9** | **2.6** | **4.5** | **7.0** | **8.5** |
|------|---------|---------|---------|---------|---------|
| **SSP1** | SSP1-1.9 | SSP1-2.6 | –       | –       | –       |
| **SSP2** | –       | SSP2-2.6 | SSP2-4.5 | SSP2-7.0 | –       |
| **SSP3** | –       | –       | SSP3-4.5 | SSP3-7.0 | SSP3-8.5 |
| **SSP4** | –       | SSP4-2.6 | SSP4-4.5 | SSP4-7.0 | –       |
| **SSP5** | –       | –       | SSP5-4.5 | –       | SSP5-8.5 |

### Approximate Mapping: Forcing → Global Warming by 2100

Radiative forcing pathways translate into different **global mean temperature increases** relative to preindustrial levels (1850–1900).  
Values below are **approximate central estimates** (IPCC AR6, median climate sensitivity assumptions):

- **1.9 W/m² (SSP1-1.9)** → ~ **1.5 °C** stabilization (consistent with Paris Agreement “1.5 °C” target).  
- **2.6 W/m² (SSP1/2/4-2.6)** → ~ **2.0 °C** warming by 2100 (low-forcing pathway).  
- **4.5 W/m² (SSP2/3/4/5-4.5)** → ~ **2.5–3.0 °C** warming by 2100 (intermediate pathway).  
- **7.0 W/m² (SSP2/3/4-7.0)** → ~ **3.5–4.0 °C** warming by 2100 (high-forcing pathway).  
- **8.5 W/m² (SSP3/5-8.5)** → ~ **4.5 °C or more** warming by 2100 (very high emissions, “worst-case” reference).

---

👉 **Interpretation**:  
- Lower forcing pathways (SSP1-1.9, SSP1/2-2.6) require **early, ambitious mitigation** and typically correspond to net-zero CO₂ by mid-century.  
- Higher forcing pathways (SSP3-7.0, SSP5-8.5) assume **limited mitigation**, continued fossil fuel use, and delayed or absent net-zero transitions.  
- Intermediate cases (SSP2-4.5) represent **current policy trajectories** or delayed mitigation efforts.  

#### 3-A) Some core scenarii outside the diagonal are not evaluated (e.g. SSP1-8.5). Why?

> ✍️ You written answer here.

#### 3-B) Try to compute the SSP1-1.9, which corresponds to net zero by 2060 and optimistic scenario. Is SSP1-1.9 out of reach?

In [ ]:
# Your code here.

> ✍️ You written answer here.

#### 3-C) Is it possible to obtain a 4°C of warming in SSP1? Evaluate SSP1 under no mitigation policy with $\mu=0$ to mimic SSP1-8.5

In [ ]:
# Your code here

> ✍️ You written answer here.

#### 3-D) Is it possible to obtain a 1.5°C in pessimistic environment? Evaluate SSP5 under mitigation policy with $\mu=1$ by 2060 to mimic SSP5-1.9

In [ ]:
# Your code here

> ✍️ You written answer here.